In [1]:
import numpy as np
import scipy as scp
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
from tqdm import tqdm

def plot_histogram(data,numberofbins,label,minimum,maximum):
    bins = np.linspace(minimum, maximum, numberofbins)
    bin_probabilities = np.histogram(data,bins)[0]/len(data)
    leftedges = bins[:-1]
    rightedges = bins[1:]
    widths = np.array(rightedges)-np.array(leftedges)
    plt.bar(leftedges,bin_probabilities,width=widths,align='edge',label=label,alpha=0.5)
    binmidvalues = [(bins[i]+bins[i+1])/2 for i in range(len(bins)-1)]

In [2]:
def monthly_mortgage_calculator(principal,monthly_rate,total_loan_months = 20*12):
    if principal<=0:
        return 0
    numerator = principal*monthly_rate*((1+monthly_rate)**total_loan_months)
    denominator = ((1+monthly_rate)**total_loan_months - 1)
    return numerator/denominator

def outstanding_loan_balance(principal,monthly_rate,total_loan_months,months_paid):
    numerator = principal*( (1+monthly_rate)**total_loan_months - (1+monthly_rate)**months_paid )
    denominator = ((1+monthly_rate)**total_loan_months - 1)
    return numerator/denominator

class House(object):
    def __init__(self,price,house_type,monthly_rate,initial_paid,loan_months = 20*12):
        self.house_type = house_type
        self.price = price
        self.initial_paid = initial_paid
        self.monthly_rate = monthly_rate
        self.loan_amount = price-initial_paid
        self.loan_principal_left = price-initial_paid
        self.loan_months = loan_months
        self.months_paid = 0
        self.monthly_repayment = monthly_mortgage_calculator(price-initial_paid,monthly_rate,loan_months)
    def finish_paid(self):
        if self.months_paid == self.loan_months:
            return True
        else:
            return False
    def pay_loan(self):
        if self.finish_paid() == False:
            self.months_paid = self.months_paid + 1
            self.loan_principal_left = outstanding_loan_balance(self.loan_amount,self.monthly_rate,self.loan_months,self.months_paid)
    def get_house_type(self):
        return self.house_type
    def monthly_repayments(self):
        if self.finish_paid() == True:
            return 0
        else:
            return self.monthly_repayment
    def get_original_price(self):
        return self.price

def makes_sense_to_sell(house,current_resale_price,rental_price):
    if house.monthly_repayments()<=rental_price:#Check if rentals cover monthly repayments
        return False
    else:
        if house.get_original_price()<current_resale_price:#Check if will make profit
            return True
        else:
            return False
        
class Person(object):
    def __init__(self,salary):
        self.salary = salary
        self.savings = 0
        self.HDB_properties = []
        self.condo_properties = []
        self.agent_type = 1 #1: Household no property, 2: Household with HDB no condo, 3: Household with single condo, 4: Household with multiple property
        self.selling_HDB = False
        self.selling_condo = False
        self.buying_HDB = False
        self.buying_condo = False
    def owns_HDB(self):
        if len(self.HDB_properties) == 0:
            return False
        else:
            return True
    def get_savings(self):
        return self.savings
    def owns_condo(self):
        if len(self.condo_properties) == 0:
            return False
        else:
            return True
    def add_HDB(self,the_HDB):
        if self.owns_HDB() == True:
            raise Exception('Cannot add HDB to someone already owning HDB')
        elif self.owns_condo() == True:
            raise Exception('Cannot add HDB to someone owning condo')
        elif the_HDB.get_house_type() != 'HDB':
            raise Exception('This is not a HDB!')
        else:
            self.HDB_properties.append(the_HDB)
    def add_condo(self,the_condo):
        if the_condo.get_house_type() != 'condo':
            raise Exception('This is not a condo!')
        else:
            self.HDB_properties.append(the_condo)
    def count_property(self):
        return len(self.HDB_properties)+len(self.condo_properties)
    def calculate_income(self,rental_price):
        income = self.salary
        expenses = 0
        if self.count_property()==0:
            return income
        else:
            for a in self.HDB_properties:
                expenses = expenses + a.monthly_repayments()
            for a in self.condo_properties:
                expenses = expenses + a.monthly_repayments()
            if self.count_property()>1:
                income = income + (self.count_property()-1)*rental_price
            return income-expenses
    def make_decisions(self,HDB_price,condo_price,rental_price,monthly_mortgage_rate):
        #agent_type 2: Check if can afford to buy a resale condo without selling. 
        #Then, check if can afford to buy a resale condo with selling HDB
        self.selling_HDB = False
        self.selling_condo = False
        self.buying_HDB = False
        self.buying_condo = False
        if self.agent_type == 2:
            if monthly_mortgage_calculator(condo_price-self.savings,monthly_mortgage_rate)<rental_price:#Rental covers mortgage
                self.buying_condo = True
            else:
                if monthly_mortgage_calculator(condo_price-HDB_price-self.savings,monthly_mortgage_rate)<self.income:#Can afford to upgrade to condo by selling HDB
                    #This happens in 2 steps tho, in this first step agent sells HDB
                    self.selling_HDB = True
                #Else does nothing

        elif self.agent_type == 1:#Potential buyer for both HDB and condo
            self.buying_condo = True
            self.buying_HDB = True
        elif self.agent_type == 3:
            if monthly_mortgage_calculator(condo_price-self.savings,monthly_mortgage_rate)<rental_price:#Rental covers mortgage
                self.buying_condo = True
        elif self.agent_type == 4:
            #First, check if income is negative. If negative, check if savings are 0. If so, he must sell a condo
            if self.calculate_income(rental_price)<=0:
                if self.savings <=0:
                    self.selling_condo = True
            else:#Check if he can buy condo first
                if monthly_mortgage_calculator(condo_price-self.savings,monthly_mortgage_rate)<rental_price:#Rental covers mortgage
                    self.buying_condo = True
                else:
                    #Check if selling HDB makes sense if own HDB
                    if self.owns_HDB() == True:
                        if makes_sense_to_sell(self.HDB_properties[0],HDB_price,rental_price) == True:
                            self.selling_HDB = True
                    #Check if selling any condo makes sense
                    elif self.owns_condo() == True:
                        for a in self.condo_properties:
                            if makes_sense_to_sell(a,condo_price,rental_price)==True:
                                self.selling_condo = True
                                break
    def admin_update_agent_type(self):
        if self.count_property() == 0:
            self.agent_type = 1
        elif self.count_property() == 1 and self.owns_HDB() == True:
            self.agent_type = 2
        elif self.count_property() == 1 and self.owns_condo() == True:
            self.agent_type = 3
        elif self.count_property()>=2:
            self.agent_type = 4
    def update_savings(self,rental_price):
        self.savings = self.savings + self.calculate_income(rental_price)
    def pay_loans(self):
        for a in self.HDB_properties:
            a.pay_loan()
        for a in self.condo_properties:
            a.pay_loan()
    def remove_amt_from_savings(self,amount):
        self.savings = self.savings - amount
    def calculate_max_buying_price(self,rental_price,monthly_rate,total_loan_months = 20*12):
        if self.calculate_income(rental_price)<=0:
            return 0
        max_mortgage_repayment = self.calculate_income(rental_price)
        max_principal = (max_mortgage_repayment*((1+monthly_rate)**total_loan_months - 1))/(monthly_rate*((1+monthly_rate)**total_loan_months))
        return max_principal + self.savings

            
        
            
            
        

In [3]:
a_HDB = House(price=1000000,house_type='HDB',monthly_rate=(0.04/12),initial_paid = 100000,loan_months = 20*12)
a_condo = House(price=2000000,house_type='condo',monthly_rate=(0.04/12),initial_paid = 100000,loan_months = 20*12)
a_person = Person(salary=10000)

In [4]:
a_person.add_HDB(the_HDB=a_HDB)
a_person.owns_HDB()

True

In [5]:
a_person.count_property()

1

In [6]:
a_person.add_condo(a_condo)

In [7]:
a_person.count_property()

2